# Financial Solution Accelerator: Drawing a Company Ecosystem
This accelerator will help you process Financial Annual Reports (10K filings) or even Wikipedia data about companies, using John Snow Labs Finance NLP, to extract the following information about companies:
- Information about the Company itself (`Trading Symbol`, `State`, `Address`, Contact Information) and other names the Company is known by (`alias`, `former name`).
- Other Companies mentioned in the report as `competitors`: we will also run a "Competitor check", to understand if another company is just in the ecosystem / supply chain of the company or it is really a competitor
- People (usually management and C-level) working in that company and their past experiences, including roles and companies
- `Acquisitions` events, including the acquisition dates. `Subsidiaries` mentioned.
- Temporality (`past`, `present`, `future`) and Certainty (`possible`) of events described, including `Forward-looking statements`.

The final aim of this accelerator is to help you analyze companies information...

<img src="https://github.com/JohnSnowLabs/spark-nlp-workshop/raw/master/tutorials/Certification_Trainings_JSL/Finance/data/im1.png" alt="drawing" width="600"/>

... create a graph...

<img src="https://github.com/JohnSnowLabs/spark-nlp-workshop/raw/master/tutorials/Certification_Trainings_JSL/Finance/data/img6.png" alt="drawing" width="400"/>

... and even being able to run Graph Embeddings on top of the graph you extract (for example, to infer new relations to green nodes given the grey ones in the picture);

<img src="https://github.com/JohnSnowLabs/spark-nlp-workshop/raw/master/tutorials/Certification_Trainings_JSL/Finance/data/im4.png" alt="drawing" width="400"/>

# Get Started with Databricks Partner Connect with John Snow Labs
John Snow Labs Spark Finance NLP Library, integrated in Databricks.
Ask for your license [here](https://docs.databricks.com/integrations/ml/john-snow-labs.html)

# Starting a session

In Databricks you will already have a Spark session started for you. 

If it's not the case, you only need to do:
`jsl.start(license_json_path=[your_path_to_json_license])`

#Imports

In [0]:
from johnsnowlabs import * 

In [0]:
import os
import sys
import time
import json
import neo4j
import functools 
import graphistry
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy import spatial
from neo4j import GraphDatabase
from neo4j import GraphDatabase, Driver

print('neo4j', neo4j.__version__)
print('graphistry', graphistry.__version__)

neo4j 5.1.0
graphistry 0.28.5

### Neo4j abd pygraphistry aux functions

In [0]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response


In [0]:
def update_data(query, rows, batch_size = 10000):
    # Function to handle the updating the Neo4j database in batch mode.
    # Especially for large datasets.
    total = 0
    batch = 0
    start = time.time()
    result = None

    while batch * batch_size < len(rows):

        res = conn.query(query, parameters={'rows': rows[batch*batch_size:(batch+1)*batch_size].to_dict('records')})
        total += res[0]['total']
        batch += 1
        result = {"total":total, "batches":batch, "time":time.time()-start}
        print(result)

    return result

In [0]:
def add_ners_rels(rows, batch_size=10000):
    # Adds NER nodes to the Neo4j graph as a batch job.
    query = '''
    //chunk1 NERs
    UNWIND $rows as row
    MERGE(n1:NER{name:row.chunk1}) ON CREATE SET n1.type=row.entity1
    
    //chunk2 NERs
    MERGE(n2:NER{name:row.chunk2}) ON CREATE SET n2.type=row.entity2

    //connect NERs
    WITH row, n1, n2
    MERGE (n1)-[:LINKS{relation:row.relation}]->(n2)

    WITH n1
    MATCH (n1)
    RETURN count(*) as total  
    '''

    return update_data(query, rows, batch_size)

In [0]:
uri = 'bolt://44.212.39.47:7687'
pwd = 'november-totals-staplers'
user= 'neo4j'

conn = Neo4jConnection(uri=uri, user=user , pwd=pwd)

# Relation Extraction Pipeline

### Create Generic Base Pipeline

In [0]:
def generic_base_pipeline():
    document_assembler = nlp.DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

    sentence_detector = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl","en")\
        .setInputCols(["document"])\
        .setOutputCol("sentence")

    tokenizer = nlp.Tokenizer()\
        .setInputCols(["sentence"])\
        .setOutputCol("token")

    embeddings = nlp.BertEmbeddings.pretrained("bert_embeddings_sec_bert_base","en") \
        .setInputCols(["sentence", "token"])\
        .setOutputCol("embeddings")
    
    base_pipeline = Pipeline(stages=[
        document_assembler,
        sentence_detector,
        tokenizer,
        embeddings
    ])
    
    return base_pipeline
    

In [0]:
ner_model_role = finance.NerModel.pretrained("finner_org_per_role_date", "en", "finance/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner_role")

ner_converter_role = nlp.NerConverter()\
    .setInputCols(["sentence","token","ner_role"])\
    .setOutputCol("ner_chunk_role")

ner_model_alias = finance.NerModel.pretrained("finner_orgs_prods_alias","en","finance/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner_alias")

ner_converter_alias = nlp.NerConverter()\
    .setInputCols(["sentence","token","ner_alias"])\
    .setOutputCol("ner_chunk_alias")\

ner_model_ticker = finance.NerModel.pretrained("finner_ticker", "en", "finance/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner_ticker")\

ner_converter_ticker = nlp.NerConverter()\
    .setInputCols(["sentence", "token", "ner_ticker"])\
    .setOutputCol("ner_chunk_ticker")

chunk_merger = finance.ChunkMergeApproach()\
    .setInputCols("ner_chunk_alias", "ner_chunk_role", "ner_chunk_ticker")\
    .setOutputCol('ner_chunk')\
    .setMergeOverlapping(True)\

pos = PerceptronModel.pretrained()\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("pos")

dependency_parser = DependencyParserModel().pretrained("dependency_conllu", "en")\
    .setInputCols(["sentence", "pos", "token"])\
    .setOutputCol("dependencies")

re_ner_chunk_filter_ticker = finance.RENerChunksFilter()\
    .setInputCols(["ner_chunk", "dependencies"])\
    .setOutputCol("re_ner_chunk_ticker")\
    .setRelationPairs(["ORG-TICKER"])\
    .setMaxSyntacticDistance(5)

re_ner_chunk_filter_alias = finance.RENerChunksFilter()\
    .setInputCols(["ner_chunk", "dependencies"])\
    .setOutputCol("re_ner_chunk_alias")\
    .setRelationPairs(["ORG-ALIAS"])\
    .setMaxSyntacticDistance(5)

re_ner_chunk_filter_role = finance.RENerChunksFilter()\
    .setInputCols(["ner_chunk", "dependencies"])\
    .setOutputCol("re_ner_chunk_role")\
    .setRelationPairs(["PERSON-ROLE", "ORG-ROLE", "DATE-ROLE"])\
    .setMaxSyntacticDistance(5)

re_ner_chunk_filter_acq = finance.RENerChunksFilter()\
    .setInputCols(["ner_chunk", "dependencies"])\
    .setOutputCol("re_ner_chunk_acq")\
    .setRelationPairs(["DATE-ORG", "DATE-ALIAS", "DATE-PRODUCT", "ORG-ORG"])\
    .setMaxSyntacticDistance(5)

re_model_exp = finance.RelationExtractionDLModel.pretrained("finre_work_experience", "en", "finance/models")\
    .setInputCols(["re_ner_chunk_role", "sentence"])\
    .setOutputCol("relations_role")\
    .setPredictionThreshold(0.5)

re_model_ticker = finance.RelationExtractionDLModel.pretrained("finre_has_ticker", "en", "finance/models")\
    .setInputCols(["re_ner_chunk_ticker", "sentence"])\
    .setOutputCol("relations_ticker")\
    .setPredictionThreshold(0.2)

re_model_acq = finance.RelationExtractionDLModel.pretrained("finre_acquisitions_subsidiaries", "en", "finance/models")\
    .setInputCols(["re_ner_chunk_acq", "sentence"])\
    .setOutputCol("relations_acq")\
    .setPredictionThreshold(0.5)

re_model_alias = finance.RelationExtractionDLModel().pretrained("finre_org_prod_alias", "en", "finance/models")\
    .setPredictionThreshold(0.5)\
    .setInputCols(["re_ner_chunk_alias", "sentence"])\
    .setOutputCol("relations_alias")

annotation_merger = finance.AnnotationMerger()\
    .setInputCols("relations_alias", "relations_role", "relations_acq", "relations_ticker")\
    .setInputType("ner_chunk")\
    .setOutputCol("relations")

nlpPipeline = Pipeline(stages=[
    generic_base_pipeline(),
    ner_model_role,
    ner_converter_role,
    ner_model_alias,
    ner_converter_alias,
    ner_model_ticker,
    ner_converter_ticker,
    chunk_merger,
    pos,
    dependency_parser,
    re_ner_chunk_filter_ticker,
    re_ner_chunk_filter_alias,
    re_ner_chunk_filter_role,
    re_ner_chunk_filter_acq,
    re_model_exp,
    re_model_ticker,
    re_model_acq,
    re_model_alias,
    annotation_merger
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)


finner_org_per_role_date download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]
finner_orgs_prods_alias download started this may take some time.
[ | ][ / ][OK!]
finner_ticker download started this may take some time.
[ | ][ / ][OK!]
pos_anc download started this may take some time.
Approximate size to download 3.9 MB
[ | ][ / ][ — ][OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][OK!]
finre_work_experience download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]
finre_has_ticker download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]
finre_acquisitions_subsidiaries download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]
finre_org_prod_alias download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[ | ][ / ][OK!]
bert_embeddings_sec_bert_base download started this may take some time.
Approximate size to download 390.4 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

### Get Relations As a Dataframe

In [0]:
import pandas as pd

def get_relations_df (results, col='relations'):
    rel_pairs=[]
    for rel in results[0][col]:
        rel_pairs.append((
          rel.result, 
          rel.metadata['entity1'], 
          rel.metadata['entity1_begin'],
          rel.metadata['entity1_end'],
          rel.metadata['chunk1'], 
          rel.metadata['entity2'],
          rel.metadata['entity2_begin'],
          rel.metadata['entity2_end'],
          rel.metadata['chunk2'], 
          rel.metadata['confidence']
      ))

    rel_df = pd.DataFrame(rel_pairs, columns=['relation','entity1','entity1_begin','entity1_end','chunk1','entity2','entity2_begin','entity2_end','chunk2', 'confidence'])

    return rel_df

### Sample Texts

In [0]:
sample_text = ["""On January 15, 2020, Cadence acquired all of the outstanding equity of AWR Corporation ("AWR"). On February 6, 2020, Cadence also acquired all of the outstanding equity of Integrand Software Inc.""",
               
"""Davin W. Cushman has been a director since November 2018. Mr. Cushman currently serves as CEO of Brightrose Software, a private, acquisition-focused growth company launched in 2021 from Cushman Management Company.""",
              
"""MTH - Meritage Homes Corporation reports disappointing revenue. RECN - Resources Connection Inc. shareholder Raymond James Trust has decreased holding."""]
            

### Get Result

In [0]:
light_model = LightPipeline(model)

rel_df = pd.DataFrame()

for i in range(len(sample_text)):
    result = light_model.fullAnnotate(sample_text[i])
    rel_df = pd.concat([rel_df,get_relations_df(result)],axis = 0,ignore_index=True)

rel_df[rel_df["relation"] != "no_rel"]

Out[13]:

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
0,has_alias,ORG,71,85,AWR Corporation,ALIAS,89,91,AWR,0.94383454
1,was_acquired,DATE,3,18,"January 15, 2020",ORG,21,27,Cadence,0.9389733
2,was_acquired,DATE,99,114,"February 6, 2020",ORG,117,123,Cadence,0.9178721
3,was_acquired,DATE,99,114,"February 6, 2020",ORG,172,193,Integrand Software Inc,0.8110341
4,was_acquired_by,ORG,117,123,Cadence,ORG,172,193,Integrand Software Inc,0.8877777
5,has_role,PERSON,0,15,Davin W. Cushman,ROLE,28,35,director,0.93061197
6,has_role_from,ROLE,28,35,director,DATE,43,55,November 2018,0.93675715
7,has_role,PERSON,58,68,Mr. Cushman,ROLE,90,92,CEO,0.75338876
8,has_role_in_company,ROLE,90,92,CEO,ORG,97,115,Brightrose Software,0.9716381
9,has_role_from,ROLE,90,92,CEO,DATE,176,179,2021,0.87006676


### Visualize Results

In [0]:
from sparknlp_display import RelationExtractionVisualizer

re_vis = viz.RelationExtractionVisualizer()

for i in range(len(sample_text)):
    result = light_model.fullAnnotate(sample_text[i])
    displayHTML(re_vis.display(result = result[0], relation_col = "relations", document_col = "document", exclude_relations = ["no_rel"], return_html=True, show_relations=True))

On January 15, 2020 DATE , Cadence ORG acquired all of the outstanding equity of AWR Corporation ORG (" AWR ALIAS "). On February 6, 2020 DATE , Cadence ORG also acquired all of the outstanding equity of Integrand Software Inc ORG . <polyline fill="none" points="372.0,155.0 370.56122448979585,153.7960798646822 369.12244897959175,152.59312871337622 367.68367346938777,151.39211553009378 366.2448979591837,150.19400929884657 364.8061224489795,148.99977900364644 363.3673469387754,147.81039362850512 361.9285714285715,146.62682215743445 360.4897959183674,145.45003357444605 359.0510204081633,144.28099686355176 357.61224489795916,143.12068100876337 356.17346938775506,141.97005499409258 354.73469387755097,140.83008780355124 353.29591836734687,139.70174842115105 351.8571428571429,138.5860058309038 350.41836734693885,137.48382901682123 348.97959183673476,136.39618696291512 347.5408163265307,135.32404865319725 346.1020408163265,134.2683830716793 344.6632653061224,133.23015920237316 343.22448979591826,132.2103460292905 341.7857142857142,131.20991253644314 340.34693877551024,130.22982770784282 338.90816326530614,129.2710605275013 337.4693877551021,128.33457997943034 336.03061224489795,127.42135504764173 334.59183673469386,126.53235471614718 333.15306122448976,125.6685479689585 331.7142857142857,124.83090379008749 330.2755102040816,124.02039116354581 328.8367346938776,123.2379790733453 327.3979591836735,122.48463650349771 325.95918367346934,121.76133243801479 324.5204081632653,121.06903586090831 323.0816326530612,120.40871575619002 321.64285714285717,119.78134110787173 320.204081632653,119.18788089996517 318.765306122449,118.62930411648207 317.32653061224494,118.10657974143427 315.88775510204084,117.62067675883348 314.44897959183675,117.17256415269148 313.01020408163265,116.76321090702004 311.57142857142856,116.39358600583091 310.13265306122446,116.06465843313586 308.69387755102036,115.77739717294665 307.2551020408163,115.53277120927505 305.8163265306123,115.33174952613282 304.3775510204082,115.17530110753174 302.9387755102041,115.06439493748353 301.5,115.0 301.5,115.0 300.06122448979585,114.9827622844223 298.6224489795918,115.01203580140927 297.1836734693878,115.08685156694916 295.7448979591837,115.20624059703015 294.3061224489796,115.36923390764053 292.8673469387755,115.57486251476851 291.4285714285715,115.82215743440234 289.98979591836735,116.11014968253025 288.55102040816325,116.43787027514047 287.11224489795916,116.80435022822121 285.6734693877551,117.20862055776078 284.23469387755097,117.64971227974738 282.79591836734687,118.12665641016923 281.3571428571429,118.63848396501459 279.91836734693885,119.18422596027166 278.47959183673476,119.7629134119287 277.04081632653066,120.37357733597398 275.6020408163265,121.01524874839565 274.16326530612247,121.68695866518203 272.7244897959183,122.3877381023213 271.2857142857143,123.11661807580174 269.8469387755102,123.87262960161156 268.40816326530614,124.654803695739 266.96938775510205,125.46217137417233 265.53061224489795,126.29376365289971 264.09183673469386,127.14861154790944 262.6530612244898,128.02574607518974 261.2142857142857,128.92419825072886 259.7755102040817,129.842999090515 258.3367346938776,130.7811796105364 256.8979591836735,131.73777082678137 255.45918367346934,132.71180375523804 254.0204081632653,133.7023094118947 252.58163265306126,134.7083188127396 251.14285714285717,135.72886297376095 249.70408163265307,136.76297291094698 248.26530612244898,137.80967964028594 246.82653061224488,138.86801417776604 245.38775510204084,139.9370075393756 243.94897959183675,141.01569074110276 242.51020408163268,142.1030947989358 241.07142857142858,143.19825072886297 239.63265306122446,144.30018954687245 238.1938775510204,145.40794226895255 236.75510204081633,146.52053991109145 235.31632653061226,147.63701348927742 233.87755102040816,148.75639401949866 232.43877551020407,149.87771251774345 231.0,151.0" stroke="#0511C1" stroke-width="1" /> was_acquired <polyline fill="none" points="274.0,55.0 272.571428571

Davin W. Cushman PERSON has been a director ROLE since November 2018 DATE . Mr. Cushman PERSON currently serves as CEO ROLE of Brightrose Software ORG , a private, acquisition-focused growth company launched in 2021 DATE from Cushman Management Company ORG . <polyline fill="none" points="41.0,255.0 40.93877551020408,254.33673469387753 40.877551020408156,253.67346938775506 40.81632653061225,253.0102040816327 40.755102040816325,252.3469387755102 40.69387755102041,251.68367346938777 40.63265306122449,251.02040816326527 40.571428571428584,250.35714285714292 40.51020408163266,249.69387755102048 40.44897959183673,249.03061224489795 40.38775510204081,248.36734693877548 40.32653061224489,247.70408163265301 40.265306122448976,247.04081632653057 40.20408163265306,246.37755102040813 40.14285714285714,245.71428571428572 40.081632653061234,245.0510204081633 40.020408163265316,244.38775510204087 39.9591836734694,243.7244897959184 39.897959183673464,243.0612244897959 39.83673469387755,242.3979591836735 39.77551020408163,241.734693877551 39.71428571428571,241.07142857142856 39.65306122448979,240.40816326530614 39.59183673469388,239.74489795918367 39.530612244897966,239.08163265306123 39.46938775510204,238.41836734693877 39.40816326530612,237.75510204081633 39.3469387755102,237.0918367346939 39.28571428571429,236.42857142857144 39.224489795918366,235.76530612244898 39.16326530612245,235.10204081632654 39.102040816326536,234.43877551020412 39.04081632653061,233.77551020408163 38.9795918367347,233.11224489795916 38.91836734693877,232.44897959183675 38.85714285714286,231.7857142857143 38.79591836734694,231.12244897959184 38.734693877551024,230.4591836734694 38.673469387755105,229.79591836734696 38.61224489795919,229.13265306122452 38.55102040816327,228.46938775510205 38.48979591836735,227.80612244897958 38.42857142857143,227.14285714285717 38.36734693877551,226.4795918367347 38.30612244897959,225.81632653061223 38.244897959183675,225.15306122448982 38.183673469387756,224.48979591836735 38.12244897959184,223.8265306122449 38.06122448979592,223.16326530612244 38.0,222.5 38.0,222.5 37.93877551020408,221.83673469387753 37.87755102040816,221.17346938775506 37.816326530612244,220.51020408163268 37.755102040816325,219.84693877551018 37.69387755102041,219.18367346938777 37.63265306122449,218.52040816326527 37.57142857142858,217.85714285714292 37.510204081632665,217.19387755102045 37.44897959183674,216.53061224489798 37.38775510204081,215.86734693877548 37.3265306122449,215.20408163265301 37.265306122448976,214.54081632653055 37.20408163265306,213.87755102040816 37.14285714285714,213.21428571428572 37.081632653061234,212.55102040816328 37.02040816326531,211.8877551020408 36.9591836734694,211.22448979591837 36.89795918367347,210.5612244897959 36.83673469387755,209.89795918367346 36.77551020408163,209.23469387755094 36.71428571428571,208.57142857142853 36.65306122448979,207.90816326530611 36.59183673469387,207.24489795918365 36.53061224489796,206.58163265306123 36.46938775510204,205.91836734693874 36.40816326530612,205.25510204081633 36.3469387755102,204.59183673469386 36.28571428571429,203.92857142857144 36.224489795918366,203.26530612244898 36.16326530612245,202.6020408163265 36.102040816326536,201.93877551020407 36.04081632653061,201.27551020408163 35.9795918367347,200.6122448979592 35.91836734693877,199.94897959183672 35.85714285714286,199.2857142857143 35.79591836734694,198.62244897959184 35.734693877551024,197.9591836734694 35.673469387755105,197.29591836734693 35.61224489795919,196.6326530612245 35.55102040816327,195.96938775510205 35.48979591836735,195.30612244897958 35.42857142857143,194.64285714285717 35.36734693877551,193.9795918367347 35.30612244897959,193.31632653061223 35.244897959183675,192.65306122448982 35.183673469387756,191.98979591836735 35.12244897959184,191.32653061224488 35.06122448979592,190.66326530612247 35.0,190.0" stroke="#000919" stroke-width="1" /> has_role_from <polyline fill="none" points="216.0,155.0 214.45918367346934,153.7

MTH TICKER - Meritage Homes Corporation ORG reports disappointing revenue. RECN TICKER - Resources Connection Inc ORG . shareholder Raymond James Trust has decreased holding. <polyline fill="none" points="221.0,55.0 219.26530612244898,53.79607986468223 217.53061224489792,52.593128713376224 215.79591836734696,51.39211553009376 214.06122448979593,50.194009298846574 212.3265306122449,48.999779003646445 210.59183673469386,47.810393628505125 208.8571428571429,46.626822157434425 207.12244897959187,45.450033574446024 205.38775510204084,44.280996863551756 203.6530612244898,43.12068100876335 201.91836734693877,41.97005499409259 200.18367346938774,40.83008780355124 198.44897959183675,39.70174842115105 196.71428571428572,38.586005830903794 194.97959183673476,37.48382901682123 193.2448979591837,36.396186962915124 191.51020408163268,35.32404865319723 189.77551020408163,34.26838307167932 188.04081632653066,33.23015920237316 186.3061224489796,32.21034602929051 184.57142857142858,31.209912536443145 182.83673469387756,30.229827707842823 181.10204081632654,29.271060527501298 179.3673469387755,28.334579979430348 177.6326530612245,27.421355047641715 175.89795918367346,26.532354716147186 174.16326530612247,25.668547968958514 172.42857142857147,24.83090379008747 170.69387755102042,24.020391163545803 168.9591836734694,23.237979073345294 167.2244897959184,22.484636503497697 165.48979591836735,21.761332438014776 163.75510204081633,21.069035860908297 162.0204081632653,20.408715756190023 160.2857142857143,19.781341107871725 158.55102040816328,19.187880899965155 156.8163265306123,18.629304116482082 155.08163265306123,18.106579741434267 153.34693877551024,17.620676758833483 151.61224489795921,17.172564152691482 149.87755102040816,16.76321090702004 148.14285714285717,16.393586005830905 146.40816326530611,16.064658433135854 144.67346938775512,15.77739717294665 142.9387755102041,15.53277120927505 141.20408163265307,15.33174952613282 139.46938775510205,15.175301107531727 137.73469387755102,15.064394937483533 136.0,15.0 136.0,15.0 134.26530612244898,14.982762284422305 132.53061224489792,15.012035801409274 130.79591836734693,15.086851566949145 129.0612244897959,15.20624059703015 127.3265306122449,15.369233907640522 125.59183673469386,15.574862514768505 123.8571428571429,15.822157434402333 122.12244897959185,16.11014968253024 120.38775510204083,16.43787027514046 118.65306122448976,16.80435022822123 116.91836734693875,17.208620557760792 115.18367346938774,17.64971227974738 113.44897959183673,18.126656410169232 111.71428571428571,18.638483965014576 109.9795918367347,19.184225960271654 108.24489795918367,19.762913411928704 106.51020408163266,20.37357733597396 104.77551020408163,21.01524874839565 103.0408163265306,21.686958665182026 101.30612244897956,22.387738102321304 99.57142857142857,23.116618075801746 97.83673469387755,23.87262960161157 96.10204081632654,24.654803695739016 94.36734693877551,25.462171374172325 92.63265306122449,26.293763652899724 90.89795918367346,27.148611547909454 89.16326530612244,28.02574607518976 87.42857142857143,28.92419825072886 85.69387755102039,29.842999090515008 83.9591836734694,30.78117961053642 82.22448979591837,31.737770826781354 80.48979591836735,32.71180375523804 78.75510204081633,33.7023094118947 77.0204081632653,34.70831881273959 75.28571428571429,35.728862973760926 73.55102040816327,36.762972910946964 71.81632653061224,37.809679640285935 70.08163265306122,38.868014177766064 68.34693877551021,39.93700753937559 66.61224489795919,41.015690741102766 64.87755102040816,42.103094798935814 63.142857142857146,43.19825072886297 61.408163265306115,44.300189546872474 59.67346938775511,45.40794226895255 57.93877551020409,46.52053991109146 56.20408163265307,47.63701348927743 54.46938775510204,48.75639401949867 52.73469387755102,49.877712517743454 51.0,51.0" stroke="#C7BBA5" stroke-width="1" /> has_ticker <polyline fill="none" points="138.0,155.0 143.5816326530612,154.33673469387753 149.16326530612247,153.6734693877551 154.7448979591837,153.

In [0]:
const_ners = 'CREATE CONSTRAINT ners IF NOT EXISTS ON (n:NER) ASSERT n.name IS UNIQUE'
conn.query(const_ners)

Out[15]: []

In [0]:
delete_all_nodes = 'MATCH (n) DETACH DELETE n;'
conn.query(delete_all_nodes)

Out[16]: []

In [0]:
add_ners_rels(rel_df)

{'total': 14, 'batches': 1, 'time': 0.09171867370605469}
Out[17]: {'total': 14, 'batches': 1, 'time': 0.09171867370605469}

![graph.png](/files/FINLEG/graph_relation.png)

# Assertion Status Pipeline for Identify COMPETITORS

In [0]:
ner = finance.NerModel.pretrained("finner_orgs_prods_alias", "en", "finance/models")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = nlp.NerConverter()\
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")\
    .setWhiteList(['ORG', 'PRODUCT'])

assertion = finance.AssertionDLModel.pretrained("finassertion_competitors", "en", "finance/models")\
    .setInputCols(["sentence", "ner_chunk", "embeddings"])\
    .setOutputCol("assertion")

nlpPipeline = Pipeline(stages=[
    generic_base_pipeline(),
    ner,
    ner_converter,
    assertion
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

light_model = LightPipeline(model)

finner_orgs_prods_alias download started this may take some time.
[ | ][OK!]
finassertion_competitors download started this may take some time.
[ | ][ / ][ — ][OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[ | ][OK!]
bert_embeddings_sec_bert_base download started this may take some time.
Approximate size to download 390.4 MB
[ | ][OK!]

### Get Result

In [0]:
sample_text = ["""In the rapidly evolving “Ideation” market, certain elements of our application compete with Microsoft, Google, InFocus, Bluescape, Mersive, Barco, Nureva and Prysm. But, Oracle  and IBM are out of our league."""]

chunks=[]
entities=[]
status=[]


light_result = light_model.fullAnnotate(sample_text)[0]

for n,m in zip(light_result['ner_chunk'],light_result['assertion']):
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    status.append(m.result)

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

In [0]:
df

Out[24]:

,chunks,entities,assertion
0,Microsoft,ORG,COMPETITOR
1,Google,ORG,COMPETITOR
2,InFocus,ORG,COMPETITOR
3,Bluescape,ORG,COMPETITOR
4,Mersive,ORG,COMPETITOR
5,Barco,ORG,COMPETITOR
6,Nureva,ORG,COMPETITOR
7,Prysm,ORG,COMPETITOR
8,Oracle,ORG,NO_COMPETITOR
9,IBM,ORG,NO_COMPETITOR


### Visualize Assertion Result

In [0]:
vis = viz.AssertionVisualizer()

vis.set_label_colors({'COMPETITOR':'#008080', 'NO_COMPETITOR':'#800080'})
    
light_result = light_model.fullAnnotate(sample_text)[0]

displayHTML(vis.display(light_result, 'ner_chunk', 'assertion', return_html=True))


In the rapidly evolving “Ideation” market, certain elements of our application compete with Microsoft ORG COMPETITOR , Google ORG COMPETITOR , InFocus ORG COMPETITOR , Bluescape ORG COMPETITOR , Mersive ORG COMPETITOR , Barco ORG COMPETITOR , Nureva ORG COMPETITOR and Prysm ORG COMPETITOR . But, Oracle ORG NO_COMPETITOR and IBM ORG NO_COMPETITOR are out of our league.

In [0]:
def add_ners_assertion(rows, batch_size=10000):
    # Adds NER nodes to the Neo4j graph as a batch job.
    query = '''
    //chunk1 NERs
    UNWIND $rows as row
    MERGE(n1:NER{name:row.chunks}) ON CREATE SET n1.type=row.chunks
    
    //chunk2 NERs
    MERGE(n2:NER{name:row.entities}) ON CREATE SET n2.type=row.entities

    //connect NERs
    WITH row, n1, n2
    MERGE (n1)-[:LINKS{relation:row.assertion}]->(n2)

    WITH n1
    MATCH (n1)
    RETURN count(*) as total  
    '''

    return update_data(query, rows, batch_size)

In [0]:
delete_all_nodes = 'MATCH (n) DETACH DELETE n;'
conn.query(delete_all_nodes)

Out[27]: []

In [0]:
const_ners = 'CREATE CONSTRAINT ners IF NOT EXISTS ON (n:NER) ASSERT n.name IS UNIQUE'
conn.query(const_ners)

Out[28]: []

In [0]:
add_ners_assertion(df)

{'total': 10, 'batches': 1, 'time': 0.3539714813232422}
Out[29]: {'total': 10, 'batches': 1, 'time': 0.3539714813232422}

![graph.png](/files/FINLEG/graph_assertion1.png)

# Assertion Status Pipeline for Temporality/Certainty

In [0]:
ner_model_role = finance.NerModel.pretrained("finner_org_per_role_date", "en", "finance/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner_role")

ner_converter_role = nlp.NerConverter()\
    .setInputCols(["sentence","token","ner_role"])\
    .setOutputCol("ner_chunk_role")

ner_model_alias = finance.NerModel.pretrained("finner_orgs_prods_alias","en","finance/models")\
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner_alias")

ner_converter_alias = nlp.NerConverter()\
    .setInputCols(["sentence","token","ner_alias"])\
    .setOutputCol("ner_chunk_alias")\

chunk_merger = finance.ChunkMergeApproach()\
    .setInputCols("ner_chunk_alias", "ner_chunk_role")\
    .setOutputCol('ner_chunk')\
    .setMergeOverlapping(True)\

assertion = finance.AssertionDLModel.pretrained("finassertion_time", "en", "finance/models")\
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")\
    .setMaxSentLen(1200)

pipeline = Pipeline(stages=[
    generic_base_pipeline(),
    ner_model_role,
    ner_converter_role,
    ner_model_alias,
    ner_converter_alias,
    chunk_merger,
    assertion
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

light_model = LightPipeline(model)

finner_org_per_role_date download started this may take some time.
[ | ][OK!]
finner_orgs_prods_alias download started this may take some time.
[ | ][OK!]
finassertion_time download started this may take some time.
[ | ][ / ][ — ][OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[ | ][OK!]
bert_embeddings_sec_bert_base download started this may take some time.
Approximate size to download 390.4 MB
[ | ][OK!]

### Get Result

In [0]:
sample_text = ["""Sinopharm Group Co., Ltd. is a Chinese pharmaceutical company. The parent company of Sinopharm Group was Sinopharm Industrial Investment, a 51–49 joint venture of state-owned enterprise China National Pharmaceutical Group and civilian-run enterprise Fosun Pharmaceutical. Mr. Liu joined the Group since July 1992. He worked at Shanghai Pharmaceutical Station, China National Pharmaceutical Group Shanghai Co., Ltd., Shanghai Guoda Drug Chain Store Co., Ltd. and Sinopharm Holding Shenyang Co., Ltd.. Mr. Liu held senior management positions in the Company since January 2009, and is currently an executive Director, president and deputy secretary of Party Committee of the Company. Mr. Liu currently serves as the director of Sinopharm Investment and Sinopharm (CNMC LTD), the director of Sinopharm Accord, and also takes senior management positions in a number of subsidiaries. Sinopharm Group's subsidiary Sinopharm CNMC and Sinopharm Accord served as the A share counterpart of the company. However, the A share of Sinopharm Group itself was unlisted. COVID-19 Vaccine development the Sinopharm BIBP COVID-19 vaccine, also known as BBIBP-CorV, is one of two inactivated virus COVID-19 vaccines developed by Sinopharm. Peer-reviewed results published in JAMA of Phase III trials in United Arab Emirates and Bahrain showed that the BIBP-vaccine is 78.1% effective against symptomatic cases and 100% against severe cases (21 cases in vaccinated group vs. 95 cases in placebo group). In December 2020, the UAE previously announced interim results showing 86% efficacy. BIBP-CorV could be transported and stored at normal refrigerated temperatures. The BIBP-vaccine is being used in vaccination campaigns by certain countries in Asia, Africa, South America, and Europe. Sinopharm expects to produce one billion doses of the BIBP-vaccine in 2021. On 7 May 2021, the World Health Organization approved the vaccine for use in COVAX."""]

chunks=[]               
entities=[]
status=[]

light_result = light_model.fullAnnotate(sample_text)[0]

for n,m in zip(light_result['ner_chunk'], light_result['assertion']):
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    status.append(m.result)

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

In [0]:
df

Out[34]:

,chunks,entities,assertion
0,"Sinopharm Group Co., Ltd.",ORG,PRESENT
1,Sinopharm Group,ORG,PAST
2,Sinopharm Industrial Investment,ORG,PAST
3,China National Pharmaceutical Group,ORG,PAST
4,Fosun Pharmaceutical,ORG,PAST
5,Mr. Liu,PERSON,PAST
6,July 1992,DATE,PAST
7,Shanghai Pharmaceutical Station,ORG,PAST
8,China National Pharmaceutical Group Shanghai C...,ORG,PAST
9,"Shanghai Guoda Drug Chain Store Co., Ltd.",ORG,PAST


### Visualize Assertion Result

In [0]:
vis = viz.AssertionVisualizer()

light_result = light_model.fullAnnotate(sample_text)[0]

displayHTML(vis.display(light_result, 'ner_chunk', 'assertion', return_html=True))


Sinopharm Group Co., Ltd. ORG PRESENT is a Chinese pharmaceutical company. The parent company of Sinopharm Group ORG PAST was Sinopharm Industrial Investment ORG PAST , a 51–49 joint venture of state-owned enterprise China National Pharmaceutical Group ORG PAST and civilian-run enterprise Fosun Pharmaceutical ORG PAST . Mr. Liu PERSON PAST joined the Group since July 1992 DATE PAST . He worked at Shanghai Pharmaceutical Station ORG PAST , China National Pharmaceutical Group Shanghai Co., Ltd ORG PAST ., Shanghai Guoda Drug Chain Store Co., Ltd. ORG PAST and Sinopharm Holding Shenyang Co., Ltd ORG PAST .. Mr. Liu held senior management positions in the Company ORG PAST since January 2009 DATE PAST , and is currently an executive Director ROLE PAST , president ROLE PAST and deputy secretary ROLE PAST of Party Committee of the Company ORG PAST . Mr. Liu PERSON PRESENT currently serves as the director ROLE PRESENT of Sinopharm Investment ORG PRESENT and Sinopharm ORG PRESENT ( CNMC LTD ORG PRESENT ), the director ROLE PRESENT of Sinopharm Accord ORG PRESENT , and also takes senior management positions in a number of subsidiaries. Sinopharm Group's ORG PAST subsidiary Sinopharm CNMC ORG PAST and Sinopharm Accord ORG PAST served as the A share counterpart of the company. However, the A share of Sinopharm Group ORG PAST itself was unlisted. COVID-19 Vaccine PRODUCT PRESENT development the Sinopharm BIBP COVID-19 PRODUCT PRESENT vaccine, also known as BBIBP-CorV PRODUCT PRESENT , is one of two inactivated virus COVID-19 vaccines developed by Sinopharm ORG PRESENT . Peer-reviewed results published in JAMA ORG PAST of Phase III trials in United Arab Emirates and Bahrain showed that the BIBP-vaccine is 78.1% effective against symptomatic cases and 100% against severe cases (21 cases in vaccinated group vs. 95 cases in placebo group). In December 2020 DATE PAST , the UAE ORG PAST previously announced interim results showing 86% efficacy. BIBP-CorV PRODUCT POSSIBLE could be transported and stored at normal refrigerated temperatures. The BIBP-vaccine PRODUCT PRESENT is being used in vaccination campaigns by certain countries in Asia, Africa, South America, and Europe. Sinopharm ORG POSSIBLE expects to produce one billion doses of the BIBP-vaccine PRODUCT POSSIBLE in 2021 DATE POSSIBLE . On 7 May 2021 DATE PAST , the World Health Organization ORG PAST approved the vaccine for use in COVAX.

In [0]:
def add_ners_assertion(rows, batch_size=10000):
    # Adds NER nodes to the Neo4j graph as a batch job.
    query = '''
    //chunk1 NERs
    UNWIND $rows as row
    MERGE(n1:NER{name:row.chunks}) ON CREATE SET n1.type=row.chunks
    
    //chunk2 NERs
    MERGE(n2:NER{name:row.entities}) ON CREATE SET n2.type=row.entities

    //connect NERs
    WITH row, n1, n2
    MERGE (n1)-[:LINKS{relation:row.assertion}]->(n2)

    WITH n1
    MATCH (n1)
    RETURN count(*) as total  
    '''
    return update_data(query, rows, batch_size)

In [0]:
delete_all_nodes = 'MATCH (n) DETACH DELETE n;'
conn.query(delete_all_nodes)

Out[55]: []

In [0]:
const_ners = 'CREATE CONSTRAINT ners IF NOT EXISTS ON (n:NER) ASSERT n.name IS UNIQUE'
conn.query(const_ners)

Out[56]: []

In [0]:
add_ners_assertion(df)

{'total': 42, 'batches': 1, 'time': 1.4880988597869873}
Out[57]: {'total': 42, 'batches': 1, 'time': 1.4880988597869873}

![graph_assertion](/files/FINLEG/graph_assertion2.png)